In [1]:
import pandas as pd
import numpy as np

### 목적
    BM(BE / ME) 계산
    COMPUSTAT에선 Book equity
    CRSP에선 Market equity를 가져옴

### COMPUSTAT
    500개 넘는 미국 상장 회사레벨 재무데이터

### Filter -> COMPUSTAT
	1. non-financial firm
	2. standardized report
	3. domestic report (미국 상장 기준)
	4. consolidated report (통합문서)
	5. 1961.01.01 ~ 22012.12.31 데이터 사용

In [2]:
# SAS 3
compustat_permno = pd.read_csv('compustat_permno.csv')

### BE DATA
    1. SEQ/TXDB 결측치면 제거
    2. ITCB 결측치면 0으로 바꿈
    3. BVPS 결측치면 PSTKRV, PSTKL, PSTK, 0 순서로 대체
    4. BE값 minus면 제거
    5. BE값 같은 해에 여러개면 최근치만 가져옴

### Variables
    BE = Shareholder's equity(SEQ) + Deferred taxes(TXDB) + Investment tax credit(ITCB) - Book value of preferred stock(BVPS)
    SEQ: Shareholder's equity
    TXDB: Deferred taxes
    ITCB: Investment tax credit
    BVPS: Book value of preferred stock
    PSTKRV: preferred stock - redemption value
    PSTKL: preferred stock - liquidating value
    PSTK: preferred stock- par value

In [3]:
### BE data ###
compustat_permno = compustat_permno.iloc[compustat_permno['permno'].dropna().index.tolist(), :]

year = compustat_permno['datadate'].map(lambda x : str(x)[:4])
compustat_permno['year'] = year

compustat_permno['itcb'] = compustat_permno['itcb'].map(lambda x: 0 if np.isnan(x) else x )

BVPS = compustat_permno['pstkrv']
PSTKL = compustat_permno['pstkl']
PSTK = compustat_permno['pstk']

BVPS[BVPS.isnull()] = PSTKL[BVPS.isnull()]
BVPS[BVPS.isnull()] = PSTK[BVPS.isnull()]
BVPS[BVPS.isnull()] = 0

ITCB = compustat_permno['itcb']
TXDB = compustat_permno['txdb']
SEQ = compustat_permno['seq']

BE = SEQ + TXDB + ITCB - BVPS
BE = BE.map(lambda x : np.nan if x <= 0 else x)

compustat_permno['be'] = BE
BE_df = compustat_permno[['gvkey', 'datadate', 'year', 'be', 'permno', 'permco']]
BE_df = BE_df.sort_values(by = ['gvkey', 'permno', 'year', 'datadate'])
BE_df = BE_df.groupby(['gvkey', 'permno', 'year', 'datadate']).last()
BE_df = BE_df.reset_index()

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


### CRSP
    NYSE, NYSE MKT, NASDAQ, Arch exchange에 속한 US STOCK 데이터베이스임

### Filter -> CRSP
    모든 US STOCK 대상 매달의 마지막 거래일의 수정종가
    SHROUT이 0이면 결측치로 바꿈
    ALTPRC이 0이면 결측치로 바꿈
    올해 6월 market cap보고 sort한 후 다음해 5월까지가는 전략을 사용 (by fama french)
    *최근거 안쓰는 이유는 주가변동에 따라 오르락 내리락 하므로
    *12월거 안쓰는 이유는 12월에 주가가 불안정하기 때문

### ME
    1. Meq 계산 (abs(SHROUT * ALTPRC) / 1000)
    2. dlret 계산
        dlstcd 500,520,551,573,574,580,584일때 delist return은 -30% (70%회수 받는 것임)
        dlstcd 다른 code들은 -100%
    3. delist code 있을때 retadj 계산
        retadj = (1+ret)*(1+dlret)-1
    4. eretadj(초과수익률) 계산
        eretadj = retadj - rf 
    5. 한날짜에 여러개의 Meq가 있는 경우 합산하여 me값으로 만듬 

### Variables
    ME: abs(SHROUT * ALTPRC) / 1000
    PERMCO:회사 identifier 
    PERMNO:주식 identifier
    shrcd:
    exchcd:
    siccd:
    dlstcd: delist적용된 여부를 알려주는 code
    dlret: delist일때 적용이 필요한 return
    prc:월말 마지막 거래일의 주식 종가
    altprc: 매일 결측치 없는 종가 * trading없다면 minus값 나옴
    vol:
    ret: 지난달부터 이번달말까지 holding했을 때의 return
    shrout: 발행주식수
    * dlstcd 500,520,551,573,574,580,584일때 delist return은 -30% (70%회수 받는 것임)
    * dlstcd 다른 code들은 -100%
    retadj: delist적용한 수익률
    eretadj: 초과수익률(retadj - rf)

In [4]:
# SAS 5
CRSP_M = pd.read_csv('CRSP_M.csv')
year = CRSP_M['DATE'].map(lambda x: str(x)[:4])
altprc_pos = CRSP_M['ALTPRC'].map(lambda x: x if abs(x) > 0 else np.nan)

Meq_df = CRSP_M.iloc[(altprc_pos).index.tolist(), :]
Meq_df = Meq_df[Meq_df['SHROUT'] > 0]
Meq = Meq_df['ALTPRC'].map(lambda x: abs(x)) * Meq_df['SHROUT'] / 1000

dlret_df = CRSP_M.iloc[CRSP_M.iloc[CRSP_M['DLSTCD'].dropna().index.tolist()]['DLRET'].isnull().index.tolist(), :]
DLSTCD_back = dlret_df['DLSTCD'].copy()

dlret_df['DLSTCD'] = dlret_df['DLSTCD'].map(lambda x : 9999 if x in [500, 520, 574, 580, 584] else x)
dlret_df['DLSTCD'] = dlret_df['DLSTCD'].map(lambda x : 9999 if x >= 551 else x)
dlret_df['DLSTCD'] = dlret_df['DLSTCD'].map(lambda x : 9999 if x <= 573 else x)

dlret_df[dlret_df['DLSTCD'] == 9999]['DLRET'] = -0.3
dlret_df[dlret_df['DLSTCD'] != 9999]['DLRET'] = -1
dlret_df['DLSTCD'] = DLSTCD_back
dlret_df['DLRET'].dropna()

ret_df = dlret_df.loc[dlret_df['RET'].dropna().index.tolist(),:]
retadj = (1 + ret_df['RET']) * (1 + ret_df['DLRET']) - 1

dlret_df['RETADJ'] = retadj
dlret_df['RETADJ'][dlret_df[dlret_df['RET'].isnull()].index.tolist()] = dlret_df[dlret_df['RET'].isnull()]['DLRET']

CRSP_M2 = CRSP_M.copy()
CRSP_M2['RETADJ'] = dlret_df['RETADJ']
CRSP_M2['RETADJ'][CRSP_M2[CRSP_M2['DLSTCD'].isnull()]['RETADJ'].index.tolist()] = CRSP_M2[CRSP_M2['DLSTCD'].isnull()]['RET']
CRSP_M2['ERETADJ'] = CRSP_M2['RETADJ'] - CRSP_M2['rf']
CRSP_M2['Meq'] = Meq

CRSP_M3 = CRSP_M2.sort_values(by = ['DATE', 'PERMCO', 'Meq'])
ME = pd.DataFrame(CRSP_M3.groupby(['DATE', 'PERMCO', 'Meq'])['Meq'].sum())
ME.columns = ['ME']

CRSP_M3_tmp = CRSP_M3.set_index(['DATE', 'PERMCO', 'Meq'])
CRSP_M3 = pd.merge(ME, CRSP_M3_tmp, left_on = ['DATE', 'PERMCO', 'Meq'], right_on = ['DATE', 'PERMCO', 'Meq'], how = 'left')
CRSP_M3 = CRSP_M3.reset_index()

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

### BE & ME MERGE 하기
    1. ME_Jun을 생성할 때는 그해 데이터로 사용함
    2. ME_last_Dec을 생성할때는 올해 12월 값이 아닌 내년 날짜로 매치됨
        t = year(date) +1
    3. BE_last_year을 생성할때는 올해 12월 값이 아닌 내년 날짜로 매치됨
        t = year(date_ + 1
    4. ME_Jun, BE_last_year, ME_last_Dec를 merge함 (inner join)
    5. BM = BE_last_year / ME_last_Dec 로 계산
    6. ret는 7~12월은 그해의 return으로 사용, 1~6월은 다음해 꺼의 return을 가져옴 (?)
    7. ret_ME_BM은 ret와 ME_BM을 merge를 통해 match시킨 테이블임
    8. ret_ME_BM 데이터들을 shift해줌

### (중요) 데이터 사용 날짜 정리
    BM: 작년 12월에 계산한 BM이 올해 6월부터 내년 5월까지 사용됨
    ME_Jun: 올해 6월에 계산한 ME로 올해 6월부터 내년 5월까지 사용됨

### Variables
    ME_Jun: ME가 0이 아니고 6월인 데이터 (SIZE 계산용)
    ME_last_Dec: ME가 0이 아니고 12월인 데이터임 (BM 계산용)
    BE_last_year: BE가 0이 아닌 데이터 (BM 계산용)

In [5]:
# SAS 6
CRSP_M3['M'] = CRSP_M3['DATE'].map(lambda x : str(x)[4:6])

ME_Jun = CRSP_M3.loc[CRSP_M3[CRSP_M3['M'] == '06']['ME'].dropna().index.tolist(),:]
ME_Jun['t'] = ME_Jun['DATE'].map(lambda x : str(x)[:4])
ME_Jun['ME_Jun'] = ME_Jun['ME']
ME_Jun = ME_Jun[['PERMNO', 't', 'ME_Jun']]
ME_Jun = ME_Jun.sort_values(by = ['PERMNO', 't', 'ME_Jun'])

In [6]:
ME_last_Dec = CRSP_M3.loc[CRSP_M3[CRSP_M3['M'] == '12']['ME'].dropna().index.tolist(),:]
ME_last_Dec['t'] = ME_last_Dec['DATE'].map(lambda x : str(int(str(x)[:4]) + 1))
ME_last_Dec['ME_last_Dec'] = ME_last_Dec['ME']
ME_last_Dec = ME_last_Dec[['PERMNO', 't', 'ME_last_Dec']]
ME_last_Dec = ME_last_Dec.sort_values(by = ['PERMNO', 't', 'ME_last_Dec'])

In [7]:
BE_last_year = BE_df.loc[BE_df['be'].dropna().index.tolist()]
BE_last_year['t'] = BE_last_year['year'].map(lambda x: str(int(x) + 1))
BE_last_year['BE_last_year'] = BE_last_year['be']
BE_last_year = BE_last_year[['permno', 't', 'BE_last_year']]
BE_last_year = BE_last_year.sort_values(by = ['permno', 't', 'BE_last_year'])

In [8]:
BE_last_year.columns = ['PERMNO', 't', 'BE_last_year']

merge_tmp = pd.merge(ME_Jun, ME_last_Dec, how = 'inner', on = ['PERMNO','t'])
ME_BM = pd.merge(merge_tmp, BE_last_year, how = 'inner', on = ['PERMNO','t'])
ME_BM['BM'] = ME_BM['BE_last_year'] / ME_BM['ME_last_Dec']
ME_BM = ME_BM[['PERMNO', 't', 'ME_Jun', 'BM']]

In [9]:
ret = CRSP_M3.copy()
ret['M_int'] = ret['M'].map(lambda x : int(x))
ret['year'] = ret['DATE'].map(lambda x : str(x)[:4])
ret['year'][(ret[ret['M_int'] <= 6]).index.tolist()] = ret[ret['M_int'] <= 6]['year'].map(lambda x: str(int(x) - 1))
ret['t'] = ret['year']

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [10]:
ret_ME_BM = pd.merge(ret, ME_BM, how = 'left', left_on = ['PERMNO','t'],right_on = ['PERMNO','t'])
ret_ME_BM = ret_ME_BM.sort_values(['PERMNO', 'DATE'])
ret_ME_BM['altprc_lag1'] = ret_ME_BM['ALTPRC'].shift(1)
ret_ME_BM['ME_lag1'] = ret_ME_BM['ME'].shift(1)
ret_ME_BM['permno_lag1'] = ret_ME_BM['PERMNO'].shift(1)
ret_ME_BM['date_lag1'] = ret_ME_BM['DATE'].shift(1)
ret_ME_BM['date_lag1'] = ret_ME_BM['DATE'].shift(1)
ret_ME_BM['M_lag1_int'] = ret_ME_BM['date_lag1'][1:].map(lambda x: int(str(x)[4:6]))
ret_ME_BM['altprc_lag1'][(ret_ME_BM['M_int'] - ret_ME_BM['M_lag1_int']) > 1] = np.nan
ret_ME_BM['ME_lag1'][(ret_ME_BM['M_int'] - ret_ME_BM['M_lag1_int']) > 1] = np.nan
ret_ME_BM['altprc_lag1'][ret_ME_BM['PERMNO'] != ret_ME_BM['permno_lag1']] = np.nan
ret_ME_BM['ME_lag1'][ret_ME_BM['PERMNO'] != ret_ME_BM['permno_lag1']] = np.nan

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()

In [11]:
BE_df

,gvkey,permno,year,datadate,be,permco
0,1000,25881.0,1970,19701231,10.544,23369.0
1,1000,25881.0,1971,19711231,8.382,23369.0
2,1000,25881.0,1972,19721231,7.309,23369.0
3,1000,25881.0,1973,19731231,8.798,23369.0
4,1000,25881.0,1974,19741231,8.279,23369.0
...,...,...,...,...,...,...
264445,296318,13013.0,2012,20121231,2908.515,53885.0
264446,296753,13255.0,2012,20121231,NaN,53991.0
264447,296885,13707.0,2012,20121231,19.018,54281.0
264448,297209,13104.0,2011,20111231,NaN,53928.0


In [12]:
CRSP_M2

,DATE,DLSTCD,PERMNO,SHRCD,EXCHCD,SICCD,DLRET,PERMCO,PRC,VOL,RET,SHROUT,ALTPRC,rf,RETADJ,ERETADJ,Meq
0,19610131,NaN,10006,10,1,3740.0,NaN,22156,50.25,939.0,0.322368,1420.0,50.2500,0.0019,0.322368,0.320468,71.355000
1,19610131,NaN,10014,10,1,3710.0,NaN,22157,4.00,395.0,0.000000,2504.0,4.0000,0.0019,0.000000,-0.001900,10.016000
2,19610131,NaN,10030,10,1,3310.0,NaN,22160,41.75,280.0,0.087948,1627.0,41.7500,0.0019,0.087948,0.086048,67.927250
3,19610131,NaN,10057,11,1,3540.0,NaN,20020,54.00,152.0,0.142857,500.0,54.0000,0.0019,0.142857,0.140957,27.000000
4,19610131,NaN,10102,10,1,2810.0,NaN,22164,79.50,480.0,0.032468,3965.0,79.5000,0.0019,0.032468,0.030568,315.217500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921713,20121231,574.0,76999,11,3,7372.0,-0.765517,11056,NaN,123365.0,NaN,6855.0,0.3120,0.0001,-0.765517,-0.765617,2.138760
2921714,20121231,580.0,93007,11,3,9999.0,-0.774834,53201,NaN,121619.0,NaN,57097.0,0.6307,0.0001,-0.774834,-0.774934,36.011078
2921715,20121231,584.0,38790,11,2,1311.0,-0.762470,1933,NaN,21350.0,NaN,19048.0,0.3321,0.0001,-0.762470,-0.762570,6.325841
2921716,20121231,584.0,89761,11,2,3714.0,2.520000,44123,NaN,39636.0,NaN,7107.0,0.3700,0.0001,2.520000,2.519900,2.629590


In [13]:
CRSP_M3

,DATE,PERMCO,Meq,ME,DLSTCD,PERMNO,SHRCD,EXCHCD,SICCD,DLRET,PRC,VOL,RET,SHROUT,ALTPRC,rf,RETADJ,ERETADJ,M
0,19610131,74,30.56000,30.56000,NaN,17670,10,1,2080.0,NaN,32.00,159.0,0.075000,955.0,32.00,0.0019,0.075000,0.073100,01
1,19610131,267,55.54500,55.54500,NaN,18702,10,1,5810.0,NaN,35.00,225.0,-0.075908,1587.0,35.00,0.0019,-0.075908,-0.077808,01
2,19610131,301,235.22000,235.22000,NaN,68523,11,1,3830.0,NaN,95.00,860.0,0.183801,2476.0,95.00,0.0019,0.183801,0.181901,01
3,19610131,584,33.33800,33.33800,NaN,20714,10,1,5610.0,NaN,19.75,119.0,0.025974,1688.0,19.75,0.0019,0.025974,0.024074,01
4,19610131,921,25.08000,25.08000,NaN,11287,10,1,3740.0,NaN,14.25,346.0,0.117647,1760.0,14.25,0.0019,0.117647,0.115747,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2908129,20121231,54291,595.52500,595.52500,NaN,13730,11,1,2911.0,NaN,29.05,239843.0,NaN,20500.0,29.05,0.0001,NaN,NaN,12
2908130,20121231,54517,752.13600,752.13600,NaN,13656,11,3,9999.0,NaN,32.56,21929.0,0.032667,23100.0,32.56,0.0001,0.032667,0.032567,12
2908131,20121231,56028,7.97808,7.97808,NaN,16852,11,3,3670.0,NaN,5.28,105896.0,0.900649,1511.0,5.28,0.0001,0.900649,0.900549,12
2908132,20121231,56249,15102.34815,15102.34815,NaN,17279,11,1,2111.0,NaN,116.67,196351.0,-0.037058,129445.0,116.67,0.0001,-0.037058,-0.037158,12


In [14]:
ME_BM

,PERMNO,t,ME_Jun,BM
0,10000,1987,0.851594,0.210944
1,10001,1987,5.822125,1.014415
2,10001,1988,6.200000,1.207618
3,10001,1989,7.007000,1.145192
4,10001,1990,10.052250,0.818149
...,...,...,...,...
176481,93434,2011,24.998000,0.796865
176482,93434,2012,36.358170,0.569317
176483,93435,2011,28.538400,0.540183
176484,93436,2011,3028.966530,0.081921


In [15]:
ret

,DATE,PERMCO,Meq,ME,DLSTCD,PERMNO,SHRCD,EXCHCD,SICCD,DLRET,...,RET,SHROUT,ALTPRC,rf,RETADJ,ERETADJ,M,M_int,year,t
0,19610131,74,30.56000,30.56000,NaN,17670,10,1,2080.0,NaN,...,0.075000,955.0,32.00,0.0019,0.075000,0.073100,01,1,1960,1960
1,19610131,267,55.54500,55.54500,NaN,18702,10,1,5810.0,NaN,...,-0.075908,1587.0,35.00,0.0019,-0.075908,-0.077808,01,1,1960,1960
2,19610131,301,235.22000,235.22000,NaN,68523,11,1,3830.0,NaN,...,0.183801,2476.0,95.00,0.0019,0.183801,0.181901,01,1,1960,1960
3,19610131,584,33.33800,33.33800,NaN,20714,10,1,5610.0,NaN,...,0.025974,1688.0,19.75,0.0019,0.025974,0.024074,01,1,1960,1960
4,19610131,921,25.08000,25.08000,NaN,11287,10,1,3740.0,NaN,...,0.117647,1760.0,14.25,0.0019,0.117647,0.115747,01,1,1960,1960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2908129,20121231,54291,595.52500,595.52500,NaN,13730,11,1,2911.0,NaN,...,NaN,20500.0,29.05,0.0001,NaN,NaN,12,12,2012,2012
2908130,20121231,54517,752.13600,752.13600,NaN,13656,11,3,9999.0,NaN,...,0.032667,23100.0,32.56,0.0001,0.032667,0.032567,12,12,2012,2012
2908131,20121231,56028,7.97808,7.97808,NaN,16852,11,3,3670.0,NaN,...,0.900649,1511.0,5.28,0.0001,0.900649,0.900549,12,12,2012,2012
2908132,20121231,56249,15102.34815,15102.34815,NaN,17279,11,1,2111.0,NaN,...,-0.037058,129445.0,116.67,0.0001,-0.037058,-0.037158,12,12,2012,2012


In [16]:
ret_ME_BM

,DATE,PERMCO,Meq,ME,DLSTCD,PERMNO,SHRCD,EXCHCD,SICCD,DLRET,...,M_int,year,t,ME_Jun,BM,altprc_lag1,ME_lag1,permno_lag1,date_lag1,M_lag1_int
1077646,19860131,7952,16.100000,16.100000,NaN,10000,10,3,3990.0,NaN,...,1,1985,1985,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1083377,19860228,7952,11.960000,11.960000,NaN,10000,10,3,3990.0,NaN,...,2,1985,1985,NaN,NaN,-4.3750,16.100000,10000.0,19860131.0,1.0
1089119,19860331,7952,16.330000,16.330000,NaN,10000,10,3,3990.0,NaN,...,3,1985,1985,NaN,NaN,-3.2500,11.960000,10000.0,19860228.0,2.0
1094881,19860430,7952,15.172000,15.172000,NaN,10000,10,3,3990.0,NaN,...,4,1985,1985,NaN,NaN,-4.4375,16.330000,10000.0,19860331.0,3.0
1100654,19860530,7952,11.793878,11.793878,NaN,10000,10,3,3990.0,NaN,...,5,1985,1985,NaN,NaN,-4.0000,15.172000,10000.0,19860430.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2897465,20120831,53453,3006.920640,3006.920640,NaN,93436,11,3,9999.0,NaN,...,8,2012,2012,3295.55667,0.074994,27.4200,2890.945440,93436.0,20120731.0,7.0
2901195,20120928,53453,3097.004160,3097.004160,NaN,93436,11,3,9999.0,NaN,...,9,2012,2012,3295.55667,0.074994,28.5200,3006.920640,93436.0,20120831.0,8.0
2904901,20121031,53453,3200.762561,3200.762561,NaN,93436,11,3,9999.0,NaN,...,10,2012,2012,3295.55667,0.074994,29.2800,3097.004160,93436.0,20120928.0,9.0
2908602,20121130,53453,3848.005780,3848.005780,NaN,93436,11,3,9999.0,NaN,...,11,2012,2012,3295.55667,0.074994,28.1314,3200.762561,93436.0,20121031.0,10.0


In [17]:
assignment1_data = ret_ME_BM.loc[ret_ME_BM[['RETADJ', 'ME_lag1', 'ME_Jun', 'BM']].dropna().index.tolist(), :]
assignment1_data = assignment1_data[['PERMNO', 'DATE', 'year', 'EXCHCD', 'SICCD', 'RETADJ',
                  'ERETADJ', 'altprc_lag1', 'ME_lag1', 'ME_Jun', 'BM']]
assignment1_data[assignment1_data['DATE'] == 20121231]

,PERMNO,DATE,year,EXCHCD,SICCD,RETADJ,ERETADJ,altprc_lag1,ME_lag1,ME_Jun,BM
2909455,10001,20121231,2012,2,4925.0,-0.015231,-0.015331,9.5200,77.654640,82.37560,0.836094
2909456,10002,20121231,2012,3,6020.0,0.010946,0.010846,2.7499,49.404703,53.70040,2.942148
2909457,10025,20121231,2012,3,3081.0,-0.020992,-0.021092,60.5000,334.625500,240.39600,0.406675
2909458,10026,20121231,2012,3,2052.0,0.018173,0.018073,62.9000,1179.312100,1117.10820,0.476147
2909460,10032,20121231,2012,3,3670.0,0.114471,0.114371,23.1500,812.125150,986.52060,0.589484
...,...,...,...,...,...,...,...,...,...,...,...
2912289,93428,20121231,2012,3,9999.0,0.148593,0.148493,31.6300,880.421050,798.49890,0.163951
2912290,93429,20121231,2012,3,9999.0,0.007672,0.007572,29.9800,2616.414560,2415.68896,0.112264
2912291,93433,20121231,2012,3,9999.0,-0.517704,-0.517804,0.8501,39.250817,28.62168,1.187065
2912280,93434,20121231,2012,3,9999.0,0.037634,0.037534,7.4400,58.575120,36.35817,0.569317


In [18]:
pd.read_csv('assignment1_sample_data.csv ')

,permno,date,year,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,ME_Jun,BM
0,10001,20121231,2012,2,4925,-0.015231,-0.015331,9.520000,77.654644,82.375603,0.836094
1,10002,20121231,2012,3,6020,0.010946,0.010846,2.749900,49.404705,53.700400,2.942148
2,10025,20121231,2012,3,3081,-0.020992,-0.021092,60.500000,334.625500,240.395996,0.406675
3,10026,20121231,2012,3,2052,0.018173,0.018073,62.900002,1179.312129,1117.108171,0.476147
4,10032,20121231,2012,3,3670,0.114471,0.114371,23.150000,812.125137,986.520627,0.589484
...,...,...,...,...,...,...,...,...,...,...,...
3203,93428,20121231,2012,3,9999,0.148593,0.148493,31.629999,880.421027,798.498921,0.163951
3204,93429,20121231,2012,3,9999,0.007672,0.007572,29.980000,2616.414520,2415.688987,0.112264
3205,93433,20121231,2012,3,9999,-0.517704,-0.517804,0.850100,39.250816,28.621680,1.187065
3206,93434,20121231,2012,3,9999,0.037634,0.037534,7.440000,58.575120,36.358170,0.569317


### 행 갯수 파악

    The data set WORK.BE has 263854 observations and 6 variables.
    The data set WORK.CRSP_M2 has 2921718 observations and 18 variables.
    The data set WORK.CRSP_M3 has 2892990 observations and 19 variables.
    The data set WORK.ME_BM has 174230 observations and 4 variables.
    The data set WORK.RET has 2892990 observations and 20 variables.
    The data set WORK.RET_ME_BM has 2892990 observations and 22 variables.
    The data set WORK.ASSIGNMENT1_DATA has 1984019 observations and 11 variables.

In [19]:
BE_df.shape, CRSP_M2.shape, CRSP_M3.shape, ME_BM.shape, ret.shape, ret_ME_BM.shape, assignment1_data.shape

((264450, 6),
 (2921718, 17),
 (2908134, 19),
 (176486, 4),
 (2908134, 22),
 (2912633, 29),
 (2010769, 11))

### 최종 테이블

In [20]:
dataset_fisrt_25 = assignment1_data.sort_values(by = ['PERMNO', 'DATE'])[:25]

final_data = assignment1_data.reset_index()
final_data = final_data.iloc[:,1:]
final_data['year_m'] = final_data['DATE'].map(lambda x: str(x)[:6])

In [21]:
dataset_fisrt_25

,PERMNO,DATE,year,EXCHCD,SICCD,RETADJ,ERETADJ,altprc_lag1,ME_lag1,ME_Jun,BM
1185224,10001,19870731,1987,3,4920.0,0.021277,0.016677,5.8750,5.822125,5.822125,1.014415
1191557,10001,19870831,1987,3,4920.0,0.083333,0.078633,6.0000,5.946000,5.822125,1.014415
1197935,10001,19870930,1987,3,4920.0,-0.022308,-0.026808,6.5000,6.441500,5.822125,1.014415
1204350,10001,19871030,1987,3,4920.0,0.020000,0.014000,6.2500,6.200000,5.822125,1.014415
1210751,10001,19871130,1987,3,4920.0,-0.029412,-0.032912,6.3750,6.324000,5.822125,1.014415
1217116,10001,19871231,1987,3,4920.0,-0.033535,-0.037435,6.1875,6.138000,5.822125,1.014415
1223460,10001,19880129,1987,3,4920.0,0.063830,0.060930,5.8750,5.828000,5.822125,1.014415
1229786,10001,19880229,1987,3,4920.0,0.080000,0.075400,6.2500,6.200000,5.822125,1.014415
1236078,10001,19880331,1987,3,4920.0,-0.076296,-0.080696,6.7500,6.696000,5.822125,1.014415
1242351,10001,19880429,1987,3,4920.0,0.030612,0.026012,6.1250,6.076000,5.822125,1.014415


In [22]:
def calcul_stat(year_m) :
    mean = final_data[final_data['year_m'] == year_m]['ERETADJ'].mean()
    std = final_data[final_data['year_m'] == year_m]['ERETADJ'].std()
    min_ = final_data[final_data['year_m'] == year_m]['ERETADJ'].min()
    max_ = final_data[final_data['year_m'] == year_m]['ERETADJ'].max()
    num = len(final_data[final_data['year_m'] == year_m]['PERMNO'].unique())

    return([mean, std, min_, max_, num])

In [23]:
Summary_stat_tb = pd.DataFrame()

for date in ['197012', '198012', '199012', '200012', '201012'] :
    Summary_stat_tb[date] = calcul_stat(date)
    
Summary_stat_tb = Summary_stat_tb.T
Summary_stat_tb.columns = ['mean', 'std', 'min', 'max', 'N of permnos']

In [24]:
Summary_stat_tb

,mean,std,min,max,N of permnos
197012,0.080578,0.110824,-0.304200,0.707338,1661.0
198012,-0.049376,0.125745,-1.013100,2.272614,3427.0
199012,-0.001951,0.184868,-1.006000,1.994000,4104.0
200012,-0.026778,0.256853,-0.999667,3.245000,4642.0
201012,0.082232,0.140242,-0.844100,2.400847,3325.0
